![alt text](http://logosz-studio.com/images/otp-logo.png)

## We want to increase the ATM's numbers in Budapest

First we install the missing python libraries

In [4]:
%%bash
/databricks/python/bin/pip install overpy

In [5]:
%%bash
/databricks/python/bin/pip install geopandas

In [6]:
%%bash
/databricks/python/bin/pip install bokeh==0.11.0

import the python libraries

In [8]:
import overpy
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import numpy as np
from shapely.geometry import Polygon
from shapely import geometry
from sklearn.neighbors import NearestNeighbors
from bokeh.plotting import figure
from bokeh.embed import components, notebook_div, file_html
from bokeh.resources import CDN

The overpass query strings, what you can test here: https://overpass-turbo.eu/

In [10]:
hairdresser_query = """area[name="Budapest"]->.a;
           node(area.a)[shop=hairdresser];out meta;""" 

market_query = """area[name="Budapest"]->.a;
           node(area.a)[amenity=marketplace];out meta;"""

"""http://wiki.openstreetmap.org/wiki/Tag:shop%3Dnewsagent"""
newsagent_query = """area[name="Budapest"]->.a;
           node(area.a)[shop=newsagent];out meta;"""

"""http://wiki.openstreetmap.org/wiki/Tag:amenity%3Dcafe"""
cafe_query = """area[name="Budapest"]->.a;
           node(area.a)[amenity=cafe];out meta;"""

"""http://wiki.openstreetmap.org/wiki/Tag:shop%3Dbakery"""
bakery_query = """area[name="Budapest"]->.a;
           node(area.a)[shop=bakery];out meta;"""

"""http://wiki.openstreetmap.org/wiki/Tag:amenity%3Datm"""
atm_query = """area[name="Budapest"]->.a;
           node(area.a)[amenity=atm];out meta;"""


pub_query = '''area[name="Budapest"]->.a;
           node(area.a)[amenity=pub];out meta;'''

bus_stop_query = """area[name="Budapest"]->.a;
           node(area.a)[highway=bus_stop];out meta;"""

"""http://wiki.openstreetmap.org/wiki/Public_transport"""
public_transport_query = """area[name="Budapest"]->.a;
           node(area.a)[public_transport];out meta;"""


pub_cafe_bakery_query = """area[name="Budapest"]->.a;
           (node(area.a)[amenity=pub];node(area.a)[amenity=cafe];node(area.a)[shop=bakery]);

out meta;"""


budapest_boundary_query = """area[name="Budapest"];
(
  node(area);
  <;
) -> .a;
(
  rel.a["boundary"="administrative"];
);
/*added by auto repair*/
(._;>;);
/*end of auto repair*/
out meta;"""

In [11]:
def getPointCoords(row, geom, coord_type):
    #Calculates coordinates ('x' or 'y') of a Point geometry (geopandas)
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y



def get_overpass_data(query,crs,to_crs=""):
  """Return geodataframe from osm database, and change the crs """
  api = overpy.Overpass()
  result = api.query(query)
  data_list = []
  for node in result.nodes:
      tgs = node.tags
      tgs['lat'] = node.lat
      tgs['lon'] = node.lon
      data_list.append(tgs)
  df = pd.DataFrame(data_list)
  geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
  geo_gdf = gpd.GeoDataFrame(df, crs=crs, geometry=geometry)
  if to_crs != "":
    """Change coordinate reference system"""
    geo_gdf = geo_gdf.to_crs(to_crs)
    """get X,Y"""
    geo_gdf['X'] = geo_gdf.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
    geo_gdf['Y'] = geo_gdf.apply(getPointCoords, geom='geometry', coord_type='y', axis=1)
    if to_crs['init'] == 'epsg:23700':
      geo_gdf = geo_gdf.rename(columns = {'X':'EOV_X','Y':'EOV_Y'})
      
    
    
  return geo_gdf
  

Download the POI's from OSM

### Use overpy (overpass) api to download POI-s  ![alt text](http://wiki.openstreetmap.org/w/images/thumb/b/b5/Overpass_API_logo.svg/300px-Overpass_API_logo.svg.png)
http://overpass-turbo.eu/  
http://wiki.openstreetmap.org/wiki/Overpass_API

In [14]:
#
crs_overpy = {'init':'epsg:4326'}
#
EOV_crs = {'init':'epsg:23700'}
atm_gdf = get_overpass_data(atm_query,crs_overpy,EOV_crs)
#Download the POIs
pub_gdf = get_overpass_data(pub_query,crs_overpy,EOV_crs)
bakery_gdf = get_overpass_data(bakery_query,crs_overpy,EOV_crs)
cafe_gdf = get_overpass_data(cafe_query,crs_overpy,EOV_crs)
bus_stop_gdf = get_overpass_data(bus_stop_query,crs_overpy,EOV_crs)
public_transport_gdf = get_overpass_data(public_transport_query,crs_overpy,EOV_crs)


In [15]:
print("The atm datasets lenghts is {}".format(len(atm_gdf)))
print("The pub datasets lenghts is {}".format(len(pub_gdf)))
print("The bakery datasets lenghts is {}".format(len(bakery_gdf)))
print("The cafe datasets lenghts is {}".format(len(cafe_gdf)))
print("The bus_stop datasets lenghts is {}".format(len(bus_stop_gdf)))
print("The public_transport datasets lenghts is {}".format(len(public_transport_gdf)))

Plot the pubs, cafes and bakery

In [17]:
# create a new plot with a title and axis labels
p = figure(title="Pub, Cafe and Bakery map", x_axis_label='EOV_X', y_axis_label='EOV_Y',width=1000, height=1000)

# add scatters with legend and line thickness
p.scatter(pub_gdf.EOV_X, pub_gdf.EOV_Y, legend="pub", size=7,color='yellow')
p.scatter(cafe_gdf.EOV_X, cafe_gdf.EOV_Y, legend="cafe", color = "blue")
p.scatter(bakery_gdf.EOV_X,bakery_gdf.EOV_Y, legend="bakery", color='orange',alpha=0.7)
# create an html document that embeds the Bokeh plot
html = file_html(p, CDN, "my plot1")

# display this html
displayHTML(html)

Select the OTP ATMs

In [19]:

#keep the important columns
atm_gdf = atm_gdf[["operator","lat","lon","EOV_X","EOV_Y"]]
#drop nullvalues from lines where the column name is operator
atm_gdf = atm_gdf.dropna(subset = ['operator'])
#drop nullvalues from lines where the column name is EOV_X
atm_gdf = atm_gdf.dropna(subset = ['EOV_X'])
#drop nullvalues from lines where the column name is EOV_Y
atm_gdf = atm_gdf.dropna(subset = ['EOV_Y'])
print("The ATM's dataframe lenght is {}".format(len(atm_gdf)))
#select OTP's ATM
len(atm_gdf[atm_gdf.operator.str.lower().str.contains("otp")])
OTP_ATM = atm_gdf[atm_gdf.operator.str.lower().str.contains("otp")]
print("The OTP ATM's dataframe lenght is {}".format(len(OTP_ATM)))

Plot the ATMs

In [21]:
# create a new plot with a title and axis labels
p = figure(title="ATM map", x_axis_label='EOV_X', y_axis_label='EOV_Y',width=1000, height=1000)

# add scatters with legend and line thickness
p.scatter(atm_gdf.EOV_X, atm_gdf.EOV_Y, legend="ATM", size=10,color='black')
p.scatter(OTP_ATM.EOV_X,OTP_ATM.EOV_Y, legend="OTP ATM", size = 7,color = "green")

# create an html document that embeds the Bokeh plot
html = file_html(p, CDN, "my plot1")

# display this html
displayHTML(html)

Plot the ATMs, Bakerys, Pubs, Cafes

In [23]:
# create a new plot with a title and axis labels
p = figure(title="ATMs, Bakerys, Pubs, Cafes map", x_axis_label='EOV_X', y_axis_label='EOV_Y',width=1000, height=1000)

# add scatters with legend and line thickness

p.scatter(pub_gdf.EOV_X, pub_gdf.EOV_Y, legend="pub", size=7,color='yellow')
p.scatter(cafe_gdf.EOV_X, cafe_gdf.EOV_Y, legend="cafe", color = "blue")
p.scatter(bakery_gdf.EOV_X,bakery_gdf.EOV_Y, legend="bakery", color='orange',alpha=0.7)
p.scatter(atm_gdf.EOV_X, atm_gdf.EOV_Y, legend="ATM", size=10,color='black')
p.scatter(OTP_ATM.EOV_X,OTP_ATM.EOV_Y, legend="OTP ATM", size = 7,color = "green")
# create an html document that embeds the Bokeh plot
html = file_html(p, CDN, "my plot1")

# display this html
displayHTML(html)

Create the kmeans cluster function, which give back the cluster centers

In [25]:
def kmeans_cluster_centers(number_of_clusters,df):
  """Return cluster centers
  :param number_of_clusters: Numbers of cluster
  :param df: Dataframe which should contains EOV_X and EOV_Y columns
  """
  number_of_clusters = int(number_of_clusters)
  X = df.loc[:,['EOV_X','EOV_Y']]
  kmeans_df = KMeans(n_clusters=number_of_clusters, random_state=2017).fit(X)
  return kmeans_df.cluster_centers_

In [26]:
# Set the cluster number
n_cluster = int(len(OTP_ATM)*1.01)
print(n_cluster)
# Create 3 kmeans cluster 
pub_gdf_kmeans_centers = kmeans_cluster_centers(n_cluster,pub_gdf)
bakery_gdf_kmeans_centers = kmeans_cluster_centers(n_cluster,bakery_gdf)
cafe_gdf_kmeans_centers = kmeans_cluster_centers(n_cluster,cafe_gdf)

In [27]:
#Merge the 3 layer to one
df_append = np.append(cafe_gdf_kmeans_centers,bakery_gdf_kmeans_centers,axis = 0)
kmeans_centroids = np.append(df_append,pub_gdf_kmeans_centers,axis = 0)
kmeans_centroids = pd.DataFrame(kmeans_centroids,columns=[['EOV_X','EOV_Y']])
len(kmeans_centroids)

Run again the kmeans to the merged layer

In [29]:
# Set the cluster number
n_cluster = int(len(OTP_ATM)*1.01)
kmeans_kmeans_centroids = kmeans_cluster_centers(n_cluster,kmeans_centroids)
kmeans_kmeans_centroids_df = pd.DataFrame(kmeans_kmeans_centroids,columns=[['EOV_X','EOV_Y']])
len(kmeans_kmeans_centroids_df)

In [30]:
# create a new plot with a title and axis labels
p = figure(title="3 kmeans centroids and the aggregated kmens centroids", x_axis_label='EOV_X', y_axis_label='EOV_Y',width=1000, height=1000)

# add scatters with legend and line thickness

p.scatter(kmeans_centroids.EOV_X, kmeans_centroids.EOV_Y, legend="3 kmeans centroids", size=7,color='yellow')
p.scatter(kmeans_kmeans_centroids_df.EOV_X, kmeans_kmeans_centroids_df.EOV_Y, legend="'3 kmeans centroids's kmeans centroids",size = 7, color = "blue")

# create an html document that embeds the Bokeh plot
html = file_html(p, CDN, "my plot1")

# display this html
displayHTML(html)


In [31]:
kmeans_kmeans_centroids_df.head()

select the important public transport stops

In [33]:

important_public_transports = public_transport_gdf[(public_transport_gdf['subway'] == 'yes') | (public_transport_gdf['train'] == 'yes') | (public_transport_gdf['highway'] == 'bus_stop')].copy()
print(len(important_public_transports))
important_public_transports = important_public_transports.dropna(subset=['name'])
important_public_transports = important_public_transports.drop_duplicates(subset=['name'])
#Set the transport weight
important_public_transports['weight'] = 1
important_public_transports['weight'][important_public_transports['train'] == 'yes'] = 3
important_public_transports['weight'][important_public_transports['subway'] == 'yes'] = 2


print(len(important_public_transports))
#keep the important columns
important_public_transports = important_public_transports[['highway','train','subway','bus','name','EOV_X','EOV_Y','weight']]

In [34]:
def gravity_model_for_kmeans_centroids(kmeans_centroids_df,kmeans_column_name_x,kmeans_column_name_y,aim_df,aim_df_x,aim_df_y,aim_df_weight_column_name = ""):
  #we want to pull every kmeans centroid an a meaningful point like a bus stop
  #we iterate allover the kmeans centroid and pull to the another point with potential filed
  gravity_centroids = pd.DataFrame()
  
  for x_y in range(0,len(kmeans_centroids_df)):
    aim_df['square_distance_k'] = (kmeans_centroids_df[kmeans_column_name_x].iloc[x_y] - aim_df.loc[:,(aim_df_x)])**2 \
                                   + (kmeans_centroids_df[kmeans_column_name_y].iloc[x_y] - aim_df.loc[:,(aim_df_y)])**2
    #calculate the gravity
    if aim_df_weight_column_name == "":
      aim_df['weight'] = 1
      aim_df['grav'] = aim_df.loc[:,('weight')]/ aim_df.loc[:,('square_distance_k')]
    else:
      aim_df['grav'] = aim_df.loc[:,(aim_df_weight_column_name)]/ aim_df.loc[:,('square_distance_k')]
      aim_df = aim_df.rename(columns={aim_df_weight_column_name:'weight'})
    # take the first order of magnitude
    grav_df = aim_df[aim_df.grav > np.max(aim_df.grav)/10].copy()
    #print("start while")
    step = 0
    len_grav_df = len(grav_df)
    kmeans_point = geometry.Point(int(kmeans_centroids_df[kmeans_column_name_x].iloc[x_y]),int(kmeans_centroids_df[kmeans_column_name_y].iloc[x_y]))
    while (len_grav_df > 1) and (step != 10):
        #get the Vx direction vector
        grav_df['Vx'] = grav_df.loc[:,(aim_df_x)]-kmeans_point.x
        #get the Vy direction vector
        grav_df['Vy'] = grav_df.loc[:,(aim_df_y)]-kmeans_point.y
        #get the degree of direction vector
        grav_df['alfa'] = np.arctan(grav_df['Vy']/grav_df['Vx'])
        #get the Gravx potential vector 
        grav_df['Gravx'] = grav_df['grav']*np.cos(grav_df['alfa'])
        #get the Gravy potential vector 
        grav_df['Gravy'] = grav_df['grav']*np.sin(grav_df['alfa'])
        #sort values by grav potential
        grav_df = grav_df.sort_values(by='grav',ascending=False)
        #step closest for the resultant direction
        kmeans_point = geometry.Point(int(kmeans_point.x+grav_df.Vx.iloc[0]/2),int(kmeans_point.y+grav_df.Vy.iloc[0]/2))
        #recalculate the potential field
        grav_df['square_distance_k'] = ((kmeans_point.x - grav_df.loc[:,(aim_df_x)])**2 + (kmeans_point.y - grav_df.loc[:,(aim_df_y)])**2)
        grav_df['grav'] = grav_df.loc[:,('weight')]/ grav_df.loc[:,('square_distance_k')]
        grav_df = grav_df[grav_df.grav > np.max(grav_df.grav)/10].copy()
        len_grav_df = len(grav_df)
               
        step = step + 1
    gravity_centroids = gravity_centroids.append(grav_df)
  
  return gravity_centroids 
      
      
    
    
    
    

# Lets create a potential field  
![alt text](http://d.ibtimes.co.uk/en/full/1459330/gravitational-waves.png)

In [36]:
grav_centr = gravity_model_for_kmeans_centroids(kmeans_kmeans_centroids_df,'EOV_X','EOV_Y',important_public_transports,'EOV_X','EOV_Y','weight')
grav_centr.head()

In [37]:
# create a new plot with a title and axis labels
p = figure(title="Grav Model Centers", x_axis_label='EOV_X', y_axis_label='EOV_Y',width=1000, height=1000)

# add scatters with legend and line thickness

p.scatter(important_public_transports.EOV_X, important_public_transports.EOV_Y, legend="Important public transport stop", size=7,color='yellow')
p.scatter(OTP_ATM.EOV_X, OTP_ATM.EOV_Y, legend="OTP ATM", size=7,color='green')
p.scatter(kmeans_kmeans_centroids_df.EOV_X, kmeans_kmeans_centroids_df.EOV_Y, legend="'3 kmeans centroids's kmeans centroids",size = 7, color = "blue")
p.scatter(grav_centr.EOV_X, grav_centr.EOV_Y, legend="Grav centers: (PublicTransport Stop)",size = 7, color = "red")
#p.scatter(kmeans_kmeans_centroids_df.EOV_X, kmeans_kmeans_centroids_df.EOV_Y, legend="'3 kmeans centroids's kmeans centroids",size = 7, color = "blue")
# create an html document that embeds the Bokeh plot
html = file_html(p, CDN, "my plot1")

# display this html
displayHTML(html)

In [38]:
fig, ax = plt.subplots()

ax.plot(atm_gdf.EOV_X, atm_gdf.EOV_Y, 'yo')
#ax.plot(OTP_ATM.EOV_X,OTP_ATM.EOV_Y,'ro')
ax.plot(pub_gdf.EOV_X,pub_gdf.EOV_Y,'bx')
#ax.plot(important_public_transports.EOV_X,important_public_transports.EOV_Y , 'bx')
#ax.plot(kmeans_kmeans_centroids_df.EOV_X,kmeans_kmeans_centroids_df.EOV_Y , 'bx')
ax.plot(grav_centr.EOV_X,grav_centr.EOV_Y , 'ro')

#ax.plot(kmeans_kmeans_centroids[::,0],kmeans_kmeans_centroids[::,1],'go')


display(fig)

# What if we dont want to rearenge the ATMs, we just add 10% more new ATMs

Merge the pub,bakery,cafe to the one dataframe, make 11 new claster, after pull to the public transport stops by gravity model

In [41]:
paying_with_chas_points = pd.DataFrame()
print(len(paying_with_chas_points))
paying_with_chas_points = paying_with_chas_points.append(pub_gdf)
print(len(paying_with_chas_points))
paying_with_chas_points = paying_with_chas_points.append(bakery_gdf)
print(len(paying_with_chas_points))
paying_with_chas_points = paying_with_chas_points.append(cafe_gdf)
print(len(paying_with_chas_points))

calculate the distance between paying_with_cash_points and ATMs by using the NearestNeighbors method

In [43]:
paying_with_chas_points = paying_with_chas_points.reset_index(drop=True)

X=atm_gdf.loc[:,['EOV_X','EOV_Y']]
#http://scikit-learn.org/stable/modules/neighbors.html
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree',n_jobs=-1).fit(X)
distances, indices = nbrs.kneighbors(paying_with_chas_points.loc[:,['EOV_X','EOV_Y']])
paying_with_chas_points['DISTANCE_FROM_NEAREST_ATM'] = distances
paying_with_chas_points.head()

Filtering a data base distance from ATM's

In [45]:
paying_with_chas_points_further_1000meters = paying_with_chas_points[paying_with_chas_points.DISTANCE_FROM_NEAREST_ATM > 1000]

In [46]:
# 10 % of ATM number
n_cluster = len(OTP_ATM)*0.1
print(n_cluster)

In [47]:
paycash_gdf_kmeans_centers = kmeans_cluster_centers(n_cluster,paying_with_chas_points_further_1000meters)
paycash_gdf_kmeans_centers = pd.DataFrame(paycash_gdf_kmeans_centers,columns=[['EOV_X','EOV_Y']])
print("The dataset lenght is {}".format(len(paycash_gdf_kmeans_centers)))
paycash_gdf_kmeans_centers.head()

In [48]:
# create a new plot with a title and axis labels
p = figure(title="New ATMs map (just 10%) ", x_axis_label='EOV_X', y_axis_label='EOV_Y',width=1000, height=1000)

# add a line renderer with legend and line thickness
p.scatter(important_public_transports.EOV_X, important_public_transports.EOV_Y, legend="public transport", size=7,color='yellow')
p.scatter(paycash_gdf_kmeans_centers.EOV_X, paycash_gdf_kmeans_centers.EOV_Y, legend="pay with cash POI kmeans center", size = 8 ,color = "black")
p.scatter(atm_gdf.EOV_X, atm_gdf.EOV_Y, legend="OTP ATM", line_width=2,color='green',alpha=0.7)
# create an html document that embeds the Bokeh plot
html = file_html(p, CDN, "my plot1")

# display this html
displayHTML(html)

In [49]:

grav_centr2 = gravity_model_for_kmeans_centroids(paycash_gdf_kmeans_centers,'EOV_X','EOV_Y',important_public_transports,'EOV_X','EOV_Y','weight')

In [50]:
# create a new plot with a title and axis labels
p = figure(title="New ATMs map (just 10%) with Gravity centers ", x_axis_label='EOV_X', y_axis_label='EOV_Y',width=1000, height=1000)

# add a line renderer with legend and line thickness
p.scatter(important_public_transports.EOV_X, important_public_transports.EOV_Y, legend="public transport", size=8,color='yellow')
p.scatter(paycash_gdf_kmeans_centers.EOV_X, paycash_gdf_kmeans_centers.EOV_Y, legend="pay with cash POI kmeans center", size = 6 ,color = "black")
p.scatter(atm_gdf.EOV_X, atm_gdf.EOV_Y, legend="OTP ATM", line_width=2,color='green',alpha=0.7)
p.scatter(grav_centr2.EOV_X, grav_centr2.EOV_Y, legend="pay with cash POI gravity center", size = 10 ,color = "red")
# create an html document that embeds the Bokeh plot
html = file_html(p, CDN, "my plot1")

# display this html
displayHTML(html)